## IMPORTS

In [1]:
import pandas as pd
import sys
import numpy as np

sys.path.append('../../')
from utils import clustering_utils, annotation_info

In [2]:
annotations_df = pd.read_csv('../eggnog/2024_none_eggnog_compiled.annotations', sep='\t')
annotations_df

,query,seed_ortholog,evalue,score,eggNOG_OGs,max_annot_lvl,COG_category,Description,Preferred_name,GOs,...,KEGG_ko,KEGG_Pathway,KEGG_Module,KEGG_Reaction,KEGG_rclass,BRITE,KEGG_TC,CAZy,BiGG_Reaction,PFAMs
0,YF00009131.t1,5911.EAR95232,0.000000e+00,2740.1,"2BZI6@1|root,2SMM4@2759|Eukaryota,3ZB0Z@5878|C...",5878|Ciliophora,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,YF00009133.t1,5911.EAR95233,2.900000e-271,894.2,"COG5159@1|root,KOG1463@2759|Eukaryota,3ZBI5@58...",5878|Ciliophora,O,PCI/PINT associated module,-,-,...,ko:K03036,"ko03050,ko05169,map03050,map05169",M00341,-,-,"ko00000,ko00001,ko00002,ko03051",-,-,-,PCI
2,YF00009135.t1,102107.XP_008240144.1,1.900000e-05,18.2,"KOG4757@1|root,KOG4757@2759|Eukaryota,37QBS@33...",35493|Streptophyta,S,Protection of telomeres protein,-,"GO:0000228,GO:0000723,GO:0000781,GO:0000782,GO...",...,-,-,-,-,-,-,-,-,-,"POT1,POT1PC"
3,YF00009136.t1,5888.CAK91573,1.400000e-18,56.9,"2EJ1D@1|root,2SPB2@2759|Eukaryota,3ZC13@5878|C...",5888.CAK91573|-,S,Forkhead associated domain,-,-,...,-,-,-,-,-,-,-,-,-,-
4,YF00009137.t1,5911.EAR95236,0.000000e+00,2276.6,"2C19C@1|root,2QPW2@2759|Eukaryota,3ZE2M@5878|C...",2759|Eukaryota,S,Viral A-type inclusion protein repeat containi...,dec-1,"GO:0000003,GO:0002064,GO:0002065,GO:0002066,GO...",...,ko:K17973,-,-,-,-,"ko00000,ko03029",-,-,-,"DEC-1_C,DEC-1_N,Dec-1"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15725,YF00013871.t1,5911.EAR97259,9.400000e-187,616.0,"COG1028@1|root,KOG1205@2759|Eukaryota,3ZCYB@58...",5878|Ciliophora,Q,KR domain,-,-,...,-,-,-,-,-,-,-,-,-,adh_short
15726,YF00013873.t1,5911.EAR97257,5.600000e-77,244.7,"2F06U@1|root,2T1DW@2759|Eukaryota,3ZF56@5878|C...",5878|Ciliophora,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
15727,YF00036611.t1,4641.GSMUA_Achr10P19120_001,1.500000e-07,26.1,"KOG1864@1|root,KOG1864@2759|Eukaryota,37Q40@33...",35493|Streptophyta,O,Belongs to the peptidase C19 family,-,-,...,ko:K11842,-,-,-,-,"ko00000,ko01000,ko01002,ko04121",-,-,-,UCH
15728,YF00013878.t1,5911.EAR97253,0.000000e+00,4710.6,"2CBV6@1|root,2QQ4Q@2759|Eukaryota,3ZBD8@5878|C...",5878|Ciliophora,S,Encoded by,-,-,...,-,-,-,-,-,-,-,-,-,-


In [3]:
full_filtered_df = pd.read_csv('../../new_raw_data/rna_seq_processed/kallisto.csv')
# full_filtered_df = full_filtered_df.rename(columns={'Unnamed: 0': 'TTHERM_ID'})
# full_filtered_norm_df = microarray_utils.normalize_expression_per_gene(full_filtered_df)

num_genes = full_filtered_df.shape[0]
num_genes

21524

In [4]:
full_filtered_df.sample(10)

,TTHERM_ID,000min,030min,060min,090min,120min,150min,180min,210min,240min
752,YF00001062.t1,0.000000,0.239048,0.475837,0.788514,1.000000,0.708114,0.638259,0.620243,0.470817
13885,YF00021553.t1,1.000000,0.638370,0.081769,0.063280,0.131608,0.355601,0.102288,0.026598,0.000000
2807,YF00004186.t1,0.119821,0.000000,0.186409,0.182457,0.399663,0.369712,0.322204,0.627002,1.000000
19576,YF00033531.t1,1.000000,0.526761,0.319099,0.238445,0.159620,0.453720,0.000000,0.166771,0.168214
10153,YF00015886.t1,0.000000,0.127835,0.360298,0.718622,1.000000,0.798134,0.568660,0.409461,0.172253
19909,YF00035237.t1,0.670583,1.000000,0.674510,0.000000,0.153046,0.139213,0.001393,0.420200,0.702058
9698,YF00015116.t1,1.000000,0.967164,0.827456,0.254731,0.707865,0.753391,0.689060,0.000000,0.180392
7250,YF00011173.t1,0.107075,0.165711,0.441228,0.757324,0.959919,1.000000,0.560044,0.195979,0.000000
18501,YF00028883.t1,1.000000,0.134124,0.392436,0.358345,0.123832,0.435568,0.000000,0.336081,0.046382
18243,YF00028485.t1,0.396410,0.071030,0.000000,0.189300,0.341799,0.261872,0.819402,0.830566,1.000000


In [5]:
cycle_expression_conflict_df = full_filtered_df.loc[
    (full_filtered_df['000min'] > 0.9)
    &
    (full_filtered_df['210min'] < 0.1)
    &
    (full_filtered_df['240min'] < 0.1)
]

In [6]:
cycle_expression_conflict_df

,TTHERM_ID,000min,030min,060min,090min,120min,150min,180min,210min,240min
24,YF00000037.t1,1.0,0.853794,0.353349,0.240808,0.223147,0.453608,0.052389,0.000000,0.062464
29,YF00000042.t1,1.0,0.554570,0.521060,0.607660,0.670361,0.924888,0.367858,0.040823,0.000000
39,YF00000055.t1,1.0,0.780745,0.826095,0.677994,0.730632,0.804309,0.292918,0.054954,0.000000
53,YF00000069.t1,1.0,0.554737,0.285618,0.122721,0.042096,0.253012,0.046195,0.000000,0.082384
60,YF00000077.t1,1.0,0.493535,0.252497,0.243926,0.242430,0.487360,0.269457,0.087669,0.000000
...,...,...,...,...,...,...,...,...,...,...
21489,YF00038282.t1,1.0,0.007389,0.040364,0.163571,0.092265,0.119939,0.076044,0.005321,0.000000
21494,YF00038289.t1,1.0,0.176599,0.306503,0.030512,0.235837,0.164901,0.029555,0.000000,0.076388
21503,YF00038319.t1,1.0,0.588812,0.030537,0.132781,0.145157,0.373829,0.099703,0.000000,0.075590
21507,YF00038342.t1,1.0,0.613393,0.650296,0.964128,0.805318,0.615479,0.285367,0.000000,0.067575


In [7]:
clustering_utils.compute_enrichment(cycle_expression_conflict_df, np.zeros(cycle_expression_conflict_df.shape[0]))

,module,term,info,fold_change,bonferroni
0,0.0,LRR_6,-,1.988023,0.004675


In [8]:
filtered_annotations_df = annotations_df.loc[annotations_df['query'].isin(list(cycle_expression_conflict_df['TTHERM_ID'].values))]
filtered_annotations_df

,query,seed_ortholog,evalue,score,eggNOG_OGs,max_annot_lvl,COG_category,Description,Preferred_name,GOs,...,KEGG_ko,KEGG_Pathway,KEGG_Module,KEGG_Reaction,KEGG_rclass,BRITE,KEGG_TC,CAZy,BiGG_Reaction,PFAMs
3,YF00009136.t1,5888.CAK91573,1.400000e-18,56.9,"2EJ1D@1|root,2SPB2@2759|Eukaryota,3ZC13@5878|C...",5888.CAK91573|-,S,Forkhead associated domain,-,-,...,-,-,-,-,-,-,-,-,-,-
13,YF00036151.t1,5911.EAR86932,1.600000e-19,57.2,"2D4M2@1|root,2SVJ4@2759|Eukaryota,3ZEY9@5878|C...",5878|Ciliophora,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
17,YF00036152.t1,5911.EAR95256,0.000000e+00,2828.9,"KOG0498@1|root,KOG0500@1|root,KOG0498@2759|Euk...",5911.EAR95256|-,U,cyclic nucleotide-gated ion channel activity,-,-,...,-,-,-,-,-,-,-,-,-,-
55,YF00016851.t1,5911.EAS04398,0.000000e+00,1552.9,"2D531@1|root,2SX70@2759|Eukaryota,3ZFR6@5878|C...",2759|Eukaryota,U,AMP-binding enzyme family protein,-,-,...,-,-,-,-,-,-,-,-,-,-
57,YF00016854.t1,5911.EAS04403,0.000000e+00,1033.3,"KOG2111@1|root,KOG2111@2759|Eukaryota",2759|Eukaryota,J,"phosphatidylinositol-3,5-bisphosphate binding",WDR45B,"GO:0000045,GO:0000322,GO:0000323,GO:0000324,GO...",...,ko:K17908,"ko04136,ko04138,ko04140,map04136,map04138,map0...",-,-,-,"ko00000,ko00001,ko02000,ko03029,ko04131",9.A.15.1,-,-,WD40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15662,YF00013764.t1,5911.EAR97362,6.100000e-253,830.6,"COG2377@1|root,2QUN9@2759|Eukaryota,3ZDZQ@5878...",5878|Ciliophora,H,Anhydro-N-acetylmuramic acid kinase,-,-,...,-,-,-,-,-,-,-,-,-,AnmK
15667,YF00013770.t1,5911.EAR97355,1.300000e-175,577.8,"COG1100@1|root,KOG0098@2759|Eukaryota",2759|Eukaryota,F,GTPase activity,-,-,...,ko:K07877,"ko04152,map04152",-,-,-,"ko00000,ko00001,ko04031,ko04131,ko04147",-,-,-,Ras
15697,YF00013820.t1,199306.XP_003069130.1,9.600000e-04,12.2,"2CRM8@1|root,2R8ER@2759|Eukaryota,38FRF@33154|...",4751|Fungi,S,ERCC4 domain,-,"GO:0000003,GO:0000075,GO:0000077,GO:0000228,GO...",...,ko:K10882,"ko03440,ko03460,map03440,map03460",-,-,-,"ko00000,ko00001,ko01000,ko03400",-,-,-,ERCC4
15710,YF00013849.t1,5911.EAR97282,3.100000e-252,830.8,"KOG3636@1|root,KOG3636@2759|Eukaryota,3ZEDW@58...",5878|Ciliophora,S,"Domain in Tre-2, BUB2p, and Cdc16p. Probable R...",-,-,...,-,-,-,-,-,-,-,-,-,RabGAP-TBC


In [9]:
terms_cols = ['COG_category', 'GOs', 'KEGG_ko', 'EC', 'PFAMs']

term_frequency = {}

for terms_col in terms_cols:
    for terms in filtered_annotations_df[terms_col].values:
        term_list = terms.split(',')
        for term in term_list:
            if term not in term_frequency:
                term_frequency[term] = 0
            term_frequency[term] += 1


In [10]:
sorted_term_frequency = dict(sorted(term_frequency.items(), key=lambda item: item[1], reverse=True))
sorted_term_frequency

{'-': 3175,
 'S': 250,
 'GO:0008150': 93,
 'GO:0005575': 92,
 'GO:0005622': 87,
 'GO:0005623': 87,
 'GO:0044424': 87,
 'GO:0044464': 87,
 'T': 84,
 'O': 81,
 'GO:0003674': 81,
 'GO:0009987': 80,
 'GO:0005737': 79,
 'GO:0043226': 76,
 'GO:0043229': 74,
 'GO:0043227': 72,
 'GO:0008152': 67,
 'GO:0043231': 67,
 'GO:0044237': 64,
 'GO:0044444': 63,
 'GO:0071704': 61,
 'GO:0044238': 59,
 'GO:0006807': 57,
 'GO:0044422': 57,
 'GO:0044446': 57,
 'GO:0065007': 52,
 'GO:0043170': 51,
 'GO:0003824': 51,
 'GO:0050789': 50,
 'GO:0005488': 49,
 'GO:0071840': 49,
 'GO:0050896': 49,
 'GO:0016043': 46,
 'LRR_6': 46,
 'Pkinase': 45,
 'U': 44,
 'GO:0050794': 44,
 'GO:1901564': 43,
 'GO:0044260': 42,
 'GO:0005634': 40,
 'GO:0032502': 39,
 'GO:0016020': 38,
 'GO:0019538': 38,
 'GO:0048856': 37,
 'GO:0044267': 36,
 'GO:0051179': 35,
 'GO:0032501': 35,
 'GO:0019222': 34,
 'GO:0032991': 33,
 'GO:0043412': 33,
 'GO:0031974': 33,
 'GO:0043233': 33,
 'GO:0070013': 33,
 'GO:0048518': 33,
 'GO:0051716': 33,
 'GO:

In [11]:
all_terms = []

all_counts = []

all_descs = []

for k, v in sorted_term_frequency.items():
    all_terms.append(k)
    all_counts.append(v)
    if k in annotation_info.go_df['GOs'].values:
       all_descs.append('; '.join(annotation_info.get_GO_info(k)))
       continue
    if k in annotation_info.kegg_df['KEGG_ko'].values:
       all_descs.append(annotation_info.get_KEGG_info(k))
       continue
    if k in annotation_info.ec_df['EC'].values:
       all_descs.append(annotation_info.get_EC_info(k))
       continue
    if k in annotation_info.COG_dict:
       all_descs.append(annotation_info.COG_dict[k])
       continue
    
    all_descs.append('-')

print(len(all_terms), len(all_counts), len(all_descs))

term_frequency_df = pd.DataFrame({
   'term': all_terms,
   'count': all_counts,
   'all_descs': all_descs
})

term_frequency_df.head(40)

3911 3911 3911


,term,count,all_descs
0,-,3175,-; -; -
1,S,250,Function unknown
2,GO:0008150,93,biological_process; A biological process is th...
3,GO:0005575,92,"cellular_component; A location, relative to ce..."
4,GO:0005622,87,intracellular anatomical structure; A componen...
5,GO:0005623,87,obsolete cell; OBSOLETE. The basic structural ...
6,GO:0044424,87,obsolete intracellular part; OBSOLETE. Any con...
7,GO:0044464,87,obsolete cell part; OBSOLETE. Any constituent ...
8,T,84,Signal transduction mechanisms
9,O,81,"Posttranslational modification, protein turnov..."
